# Imports

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import torch
import pickle
import random
import plotly.express as px
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


from config import *


In [2]:
torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

# Read Results Data

In [3]:
with open('data/predictions_dict.pickle', 'rb') as f:
    prediction_dic = pickle.load(f)

with open(BaselinePath, 'rb') as f:
    raw_dataset_dict = pickle.load(f)
    
with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)

In [4]:
def create_test_dataframe(raw_dataset_dict: dict):
    test_dict = {}
    for key in raw_dataset_dict.keys():
    #for key in ['All items']:
        df = raw_dataset_dict[key][['Category', 'Date', 'Year', 'Indent', 'Inflation t+1']]
        df.dropna(inplace=True)
        df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
        target_df = df[df['Year'] > Year]
        test_dict[key] = target_df
    return test_dict

test_dict = create_test_dataframe(raw_dataset_dict)


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_2376/1267119108.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_2376/1267119108.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_2376/1267119108.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [5]:
def get_df_with_predictions(prediction_dic: dict, dict_of_categories_df:dict) -> dict:
    all_data_dict = {}
    for key in list(prediction_dic.keys()):
        predictions = prediction_dic[key]
        prediction_df =  pd.DataFrame(predictions.flatten().detach().numpy())
        prediction_df.rename(columns = {0: 'Prediction'}, inplace=True)
        dict_of_categories_df[key] = dict_of_categories_df[key].reset_index(drop=True)
        all_data_dict[key] = pd.concat([dict_of_categories_df[key], prediction_df], axis=1)
    return all_data_dict

all_data_test_dict=get_df_with_predictions(prediction_dic, test_dict)

In [6]:
all_categories = list(all_data_test_dict.keys())

In [7]:
len(all_categories)

293

In [8]:
def avg_r_squared(all_data_test_dict):
    r_squared_lst = []
    low_r_squared_indent = []
    positive_r_squared_indent = []
    high_r_squared_indent = []
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        r2 = r2_score(y_actual, y_pred)
        r_squared_lst.append(r2)

        if r2<=0:
            low_r_squared_indent.append(df_predictions['Indent'].unique().item())
        
        else:
            positive_r_squared_indent.append(df_predictions['Indent'].unique().item())


        if r2>=0.5:
            high_r_squared_indent.append(df_predictions['Indent'].unique().item())

        if key =='All-items':
            headline_r2 = r2
    
    avg_r2_score = np.mean(r_squared_lst)
    r2_std = np.std(r_squared_lst)
    
    print(f'Average R Squared:  {avg_r2_score}')
    print(f'R Squared std:  {r2_std}')
    print(f'interval: {[avg_r2_score-r2_std, avg_r2_score+r2_std]}')
    print('--------------------------------------------------------')
    #print(f'sorted r2 list: {sorted(r_squared_lst, reverse=True)}')
    print(f'Headline R2: {headline_r2}')
    print(f'R2 list percentiles:\n[10: {np.percentile(r_squared_lst, 10)}, 25: {np.percentile(r_squared_lst, 25)}, 50: {np.percentile(r_squared_lst, 50)}, 75: {np.percentile(r_squared_lst, 75)}, 90: {np.percentile(r_squared_lst, 90)}]')
    print(f'num of neg/zero R^2: {sum(1 for i in r_squared_lst if i<= 0)}')
    print(f'num of postive R^2: {sum(1 for i in r_squared_lst if i> 0)}')
    print(f'num of high R^2: {sum(1 for i in r_squared_lst if i>= 0.5)}')


    return headline_r2, low_r_squared_indent, positive_r_squared_indent, high_r_squared_indent

In [9]:
avg_r_squared(all_data_test_dict)

Average R Squared:  -1.0498067528373343
R Squared std:  2.6637300401210604
interval: [-3.7135367929583945, 1.6139232872837261]
--------------------------------------------------------
Headline R2: -4.35158096024074
R2 list percentiles:
[10: -2.3949288353890013, 25: -0.7958368987357083, 50: -0.24569736078965065, 75: -0.008609674685224444, 90: 0.09001904017435347]
num of neg/zero R^2: 227
num of postive R^2: 66
num of high R^2: 0


(-4.35158096024074,
 [2.0,
  3.0,
  3.0,
  1.0,
  5.0,
  6.0,
  5.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  5.0,
  3.0,
  4.0,
  5.0,
  6.0,
  3.0,
  4.0,
  5.0,
  4.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  4.0,
  6.0,
  5.0,
  3.0,
  4.0,
  3.0,
  4.0,
  2.0,
  1.0,
  2.0,
  5.0,
  4.0,
  3.0,
  5.0,
  4.0,
  5.0,
  4.0,
  3.0,
  4.0,
  6.0,
  5.0,
  6.0,
  4.0,
  3.0,
  2.0,
  4.0,
  1.0,
  4.0,
  4.0,
  5.0,
  4.0,
  4.0,
  3.0,
  5.0,
  1.0,
  1.0,
  3.0,
  3.0,
  2.0,
  2.0,
  3.0,
  4.0,
  5.0,
  5.0,
  5.0,
  4.0,
  5.0,
  5.0,
  3.0,
  3.0,
  5.0,
  3.0,
  4.0,
  6.0,
  1.0,
  2.0,
  3.0,
  3.0,
  3.0,
  3.0,
  3.0,
  3.0,
  5.0,
  3.0,
  2.0,
  2.0,
  1.0,
  4.0,
  4.0,
  6.0,
  6.0,
  5.0,
  4.0,
  6.0,
  3.0,
  5.0,
  5.0,
  5.0,
  4.0,
  4.0,
  3.0,
  6.0,
  4.0,
  5.0,
  3.0,
  4.0,
  5.0,
  3.0,
  3.0,
  3.0,
  5.0,
  3.0,
  4.0,
  4.0,
  5.0,
  4.0,
  3.0,
  4.0,
  4.0,
  5.0,
  3.0,
  5.0,
  5.0,
  4.0,
  3.0,
  5.0,
  5.0,
  5.0,
  4.0,


In [10]:
def plot_results(all_data_dict, categories):
    category_samples = random.sample(categories, 10)+['All-items']
    for category in category_samples:
        category_df = all_data_dict[category]
        fig = px.line(category_df, x="Date", y=["Actual", "Prediction"], title=f'{category} - Actual VS Prediction')
        fig.show()

plot_results(all_data_test_dict, all_categories)

In [11]:
def avg_rmse(all_data_test_dict):
    mse_lst = []
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)
        mse_lst.append(mse)
    
    rmse_list = list(map(np.sqrt,mse_lst))
    avg_rmse = np.mean(rmse_list)
    rmse_std = np.std(rmse_list)
    
    print(f'RMSE:  {avg_rmse}')
    print(f'MSE std:  {rmse_std}')
    print(f'interval: {[avg_rmse-rmse_std, avg_rmse+rmse_std]}')

    return avg_rmse,rmse_std

In [12]:
avg_rmse(all_data_test_dict)

RMSE:  2.1750556329461856
MSE std:  1.760218955511989
interval: [0.4148366774341967, 3.9352745884581743]


(2.1750556329461856, 1.760218955511989)

In [13]:
def total_corr(all_data_test_dict):
    corr_dict = {}
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        corr = stats.pearsonr(y_pred,y_actual)[0]
        corr_dict[key] =  corr
    
    total_corr = sum(corr_dict.values())
    #avg_corr = total_corr/len(list(all_data_test_dict.keys()))
    
    num_high_corr = 0
    for category in corr_dict:
        if corr_dict[category] >= 0.5:
            num_high_corr +=1
    
    print(f'Number of categories with High Correlation: {num_high_corr}')
    
    return total_corr

In [14]:
total_corr(all_data_test_dict)

Number of categories with High Correlation: 13


47.13477473024962

In [16]:
df_predictions = all_data_test_dict['All-items']
y_pred = df_predictions['Prediction'].values
y_actual = df_predictions['Actual'].values
mse = mean_squared_error(y_pred, y_actual)
rmse = np.sqrt(mse)


print(f'rmse: {rmse}')

rmse: 1.0543123170603457
